In [1]:
from PIL import Image, ImageEnhance, ImageFilter
from IPython.display import Image
import matplotlib.pyplot as plt
import sys
import os
import numpy as np
import cv2
import torch
import glob
from torchvision import transforms
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.applications.resnet50 import decode_predictions
import tensorflow as tf

from src.utils_blackbox import show_image, preprocess_image, clip_eps, get_label
from src.utils_attack import compute_psnr, compute_ssim, compute_msssim, eval_psnr_ssim_msssim, bytearray_to_bits
from src.regen_pipe import ReSDPipeline
from src.blackbox import verify_watermark
from src.regen_attack import DiffWMAttacker

2025-03-05 12:10:30.097025: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-05 12:10:30.097058: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-05 12:10:30.098739: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-05 12:10:30.108683: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-05 12:10:34.270741: I tensorflow/core/common_

In [ ]:
import os
os._exit(00)


In [2]:
wm_text = 'test'
device = 'cuda:0'
print_width = 50

In [3]:
pipe = ReSDPipeline.from_pretrained("stabilityai/stable-diffusion-2-1", torch_dtype=torch.float16, variant="fp16")
pipe.set_progress_bar_config(disable=True)
pipe.to(device)
print('Finished loading model')

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 10.91 GiB of which 8.06 MiB is free. Process 3962201 has 10.90 GiB memory in use. Of the allocated memory 485.71 MiB is allocated by PyTorch, and 2.29 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
attackers = {
    'diff_attacker_60': DiffWMAttacker(pipe, batch_size=5, noise_step=60, captions={})
}

In [ ]:
attacker_name = "diff_attacker_60"
attacker = attackers[attacker_name]   # Select the attacker

print('*' * print_width)
print(f'Attacking PGD Blackbox Watermark with {attacker_name}')

# Define paths for the input and output of the attack
# watermarked_img_path = "blackbox_watermarked_image.jpg"  # Input image
watermarked_img_path = "examples/panda.jpg"
attacked_img_path = "original_panda_attacked.jpg"

# Move attacker model to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
attacker.pipe.to(device)  # Ensure the model is on the right device

# Apply the attack
attacker.attack([watermarked_img_path], [attacked_img_path])

print('Finished attacking')


In [ ]:
resnet50 = tf.keras.applications.ResNet50(weights="imagenet", include_top=True, classifier_activation=None)


In [ ]:
original_image_path = "examples/panda.jpg"
attacked_img_path = "attacked_blackbox_image.jpg"


In [ ]:
sample_image_org = show_image(original_image_path)
preprocessed_image_org = preprocess_image(sample_image_org)
image_tensor_org = tf.constant(preprocessed_image_org, dtype=tf.float32)

sample_image_att = show_image(attacked_img_path)
preprocessed_image = preprocess_image(sample_image_att)
image_tensor_att = tf.constant(preprocessed_image, dtype=tf.float32)

original_logits = resnet50.predict(preprocess_input(image_tensor_org))
attacked_logits = resnet50.predict(preprocess_input(image_tensor_att))

In [ ]:
verify_watermark(original_logits, attacked_logits, [178,728,815,136])

In [ ]:
import os
import cv2
from PIL import Image
logit_diffs_dic = {}
# Directory where the images are stored
image_dir = "./output/blackbox_epsilon_testing"

# Get a list of all images in the directory
image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))]  # Modify for other extensions if needed

# Make sure there are images to process
if not image_files:
    print("No images found in the directory.")
else:
    # Iterate over each image file
    
    for image_file in os.listdir(image_dir):
        # Construct the full image path
        image_path = os.path.join(image_dir, image_file)
        
        print(f"Processing image: {image_file}")

        watermarked_img_path = image_path
        index = image_file.index(".")
        print(image_file[8:index])
        attacked_img_path = f"./output/black_box_attacked_images/perturbed_panda_attacked{image_file[8:index]}.jpg"
        
        
        image = Image.open(watermarked_img_path).convert("RGB")
        
        attacker_name = "diff_attacker_60"
        attacker = attackers[attacker_name]   # Select the attacker
        
        print('*' * print_width)
        print(f'Attacking PGD WhiteBox Watermark with {attacker_name}')
        
        
        # Move attacker model to CUDA if available
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        attacker.pipe.to(device)  # Ensure the model is on the right device
        
        # Apply the attack
        attacker.attack([watermarked_img_path], [attacked_img_path])
        
        print(f'Finished attacking: {image_file}')

        original_image_path = "examples/panda.jpg"
        sample_image_org = show_image(original_image_path)
        preprocessed_image_org = preprocess_image(sample_image_org)
        image_tensor_org = tf.constant(preprocessed_image_org, dtype=tf.float32)
        
        sample_image_att = show_image(attacked_img_path)
        preprocessed_image = preprocess_image(sample_image_att)
        image_tensor_att = tf.constant(preprocessed_image, dtype=tf.float32)
        
        original_logits = resnet50.predict(preprocess_input(image_tensor_org))
        attacked_logits = resnet50.predict(preprocess_input(image_tensor_att))

        verif, avg_diff = verify_watermark(original_logits, attacked_logits,
                 [726, 264, 428, 190])

        dic_key = int(image_file[8:index])
        logit_diffs_dic[dic_key] = avg_diff
        
      


In [ ]:
sorted_logit_diffs_dic = {key: logit_diffs_dic[key] for key in sorted(logit_diffs_dic)}
epsilon_values = np.arange(0, 8/255, 0.5/255)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
plt.plot(epsilon_values, sorted_logit_diffs_dic.values(), marker='o', linestyle='-')
plt.xlabel("Epsilon (EPS)")
plt.ylabel("Average Logit Difference")
plt.title("Effect of Epsilon on Logit Differences after Regen Attack (Blackbox)")
plt.grid(True)
plt.show()